In [5]:
!pip install numpy pandas tensorflow scikit-learn

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,fdef cEmbedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import re
import pickle

In [11]:
np.random.seed(42)
tf.random.set_seed(42)

train_df=pd.read_csv('D:\EmojiPredictor\Train.csv')
test_df=pd.read_csv('D:\EmojiPredictor\Test.csv')
mapping_df=pd.read_csv('D:\EmojiPredictor\Mapping.csv')
emj_dict=mapping_df.set_index('number')['emoticons'].to_dict()

mapping_df.head()

,Unnamed: 0,emoticons,number
0,0,😜,0
1,1,📸,1
2,2,😍,2
3,3,😂,3
4,4,😉,4


In [12]:
train_df.head()

,Unnamed: 0,TEXT,Label
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,2,Been friends since 7th grade. Look at us now w...,2
3,3,This is what it looks like when someone loves ...,3
4,4,RT @user this white family was invited to a Bl...,3


In [13]:
test_df.head()

,Unnamed: 0,id,TEXT
0,0,0,Thought this was cool...#Repost (get_repost)・・...
1,1,1,Happy 4th! Corte madera parade. #everytownusa ...
2,2,2,Luv. Or at least something close to it. @ Unio...
3,3,3,There's a slice of pie under that whipped crea...
4,5,5,#thankyou for your thank you We adore you both...


In [50]:
def cleaning(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+|@\w+|#|[^\w\s]+', ' ', text)
    return text.strip()

train_df['TEXT']=train_df['TEXT'].apply(cleaning)
train_df=train_df.dropna(subset=['TEXT','Label'])
train_df['Label']=train_df['Label'].astype(int)
train_df= train_df.drop_duplicates(subset=['TEXT'])
test_df['TEXT']= test_df['TEXT'].apply(cleaning)
test_df =test_df.dropna(subset=['TEXT'])
train_df

,Unnamed: 0,TEXT,Label
0,0,vacation wasted vacation2017 photobomb tired v...,0
1,1,oh wynwood you re so funny wynwood art itwas3a...,1
2,2,been friends since 7th grade look at us now we...,2
3,3,this is what it looks like when someone loves ...,3
4,4,rt this white family was invited to a black ba...,3
...,...,...,...
69995,69995,yes i call galina my bubie go follow my beauti...,3
69996,69996,i sea you seattle ballard seafood festival,16
69997,69997,if one of my daughters is wearing this and ask...,2
69998,69998,guess who whoop people on their homecoming asu...,3


In [20]:
test_df

,Unnamed: 0,id,TEXT
0,0,0,thought this was cool repost get_repost colorv...
1,1,1,happy 4th corte madera parade everytownusa mer...
2,2,2,luv or at least something close to it union hi...
3,3,3,there s a slice of pie under that whipped crea...
4,5,5,thankyou for your thank you we adore you both ...
...,...,...,...
25953,7493,29993,rest in peace nana thanks for playing cards ly...
25954,7494,29994,i jus wan eat chowtime swt30 hartford connecticut
25955,7495,29995,playing bloon supermonkey 2 bloonsupermonkey2 ...
25956,7498,29998,best snow cone i ve ever had bahamabucks youha...


In [48]:
X_train_val= train_df['TEXT'].values
y_train =train_df['Label'].values
X_test_val=test_df['TEXT'].values
test_ids =test_df['id'].values
tokenizer =Tokenizer()
tokenizer.fit_on_texts(X_train_val)
word_to_index = tokenizer.word_index
X_train_seq = tokenizer.texts_to_sequences(X_train_val)
X_test_seq = tokenizer.texts_to_sequences(X_test_val)
maxlen =max(len(seq) for seq in X_train_sequences)
X_train_pad =pad_sequences(X_train_sequences,maxlen=maxlen,padding='post',truncating='post')
X_test_pad=pad_sequences(X_test_sequences,maxlen=maxlen,padding='post',truncating='post')
Y_train= to_categorical(y_train,num_classes=len(emj_dict))
X_train_val

array(['vacation wasted vacation2017 photobomb tired vacationwasted mcgar30 miami port of',
       'oh wynwood you re so funny wynwood art itwas3am flowers vibes wynwood',
       'been friends since 7th grade look at us now we all following our dreams doing what we love and',
       ...,
       'if one of my daughters is wearing this and asks me for ice cream at 2am i won t say no',
       'guess who whoop people on their homecoming asurams atlanta georgia',
       'we love you robbie heritage memorial cemetery llc'], dtype=object)

In [36]:
y_train

array([ 0,  1,  2, ...,  2,  3, 14])

In [49]:
X_test_val

array(['thought this was cool repost get_repost colorview by shay_images',
       'happy 4th corte madera parade everytownusa merica perry s on',
       'luv or at least something close to it union hill richmond virginia',
       ..., 'playing bloon supermonkey 2 bloonsupermonkey2 bloontd',
       'best snow cone i ve ever had bahamabucks youhavetotryit',
       'you making my life better is just part of your blessing'],
      dtype=object)

In [34]:
X_train_sequences

[[602, 6241, 15112, 4934, 1720, 22750, 22751, 150, 1297, 9],
 [219, 1277, 6, 93, 21, 612, 1277, 192, 22752, 1214, 497, 1277],
 [135,
  97,
  503,
  2279,
  2024,
  114,
  17,
  90,
  91,
  22,
  27,
  2458,
  35,
  654,
  421,
  57,
  22,
  14,
  7],
 [13, 15, 57, 16, 420, 54, 45, 437, 373, 6, 8501, 219, 987, 482, 4935],
 [1157,
  13,
  302,
  67,
  30,
  5749,
  4,
  5,
  425,
  5750,
  7,
  2,
  146,
  142,
  4380,
  21,
  360,
  8,
  3,
  72],
 [1278, 319, 458, 2, 33, 170, 11, 294, 69, 9, 1, 15113, 1537],
 [181, 50, 6, 4166, 66, 1947, 859, 5, 1238, 1858, 1404, 338],
 [22753, 22754, 4167, 307],
 [6242, 3772, 86, 6799, 4168, 63, 1146],
 [860,
  19,
  226,
  22755,
  4169,
  1314,
  150,
  44,
  474,
  22756,
  339,
  2118,
  20,
  4,
  673,
  20,
  8,
  3],
 [2, 14, 3, 528, 92, 179, 243, 4641, 154, 1203],
 [11822, 3, 434, 1064, 4, 5, 182, 1899, 9, 22757, 4642, 82],
 [53, 23, 196, 4, 403, 6, 22758, 22759, 22760, 7597, 223],
 [25, 332, 294, 158, 9839, 55],
 [133, 22761, 694, 1, 22762, 1

In [31]:
X_train_pad

array([[  602,  6241, 15112, ...,     0,     0,     0],
       [  219,  1277,     6, ...,     0,     0,     0],
       [  135,    97,   503, ...,     0,     0,     0],
       ...,
       [  113,    36,     9, ...,     0,     0,     0],
       [  516,   102,  9626, ...,     0,     0,     0],
       [   22,    14,     6, ...,     0,     0,     0]], dtype=int32)

In [30]:
X_test_pad

array([[  656,    13,    30, ...,     0,     0,     0],
       [   25,   332, 11508, ...,     0,     0,     0],
       [ 1206,   159,    17, ...,     0,     0,     0],
       ...,
       [  551,   124,     0, ...,     0,     0,     0],
       [   51,   708,  3952, ...,     0,     0,     0],
       [    6,   282,     3, ...,     0,     0,     0]], dtype=int32)

In [35]:
Y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
X_train,X_val,Y_train,Y_val = train_test_split(X_train_pad,Y_train,test_size=0.2,random_state=42)

In [39]:
embed_size =100
vocab_size = len(word_to_index)+1
embd_mat = np.random.uniform(low=-0.25,high=0.25,size=(vocab_size, embed_size))

model = Sequential([
    Embedding(input_dim= len(word_to_index) + 1, output_dim =embed_size, weights =[embd_mat], trainable= True), 
    LSTM(64, return_sequences =True),LSTM(32),Dense(len(emj_dict), activation ='softmax')
])
model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['accuracy'])

In [40]:
model.fit(X_train,Y_train,validation_data=(X_val,Y_val),epochs= 20, batch_size =32,verbose=1)
val_loss,val_acc =model.evaluate(X_val, Y_val,verbose=0)
print(f"Validation Accuracy: {val_acc:.4f}")

Epoch 1/20
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 220s 124ms/step - accuracy: 0.2145 - loss: 2.7461 - val_accuracy: 0.2226 - val_loss: 2.6960
Epoch 2/20
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 220s 126ms/step - accuracy: 0.2336 - loss: 2.6406 - val_accuracy: 0.2606 - val_loss: 2.5895
Epoch 3/20
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 219s 125ms/step - accuracy: 0.2873 - loss: 2.4404 - val_accuracy: 0.2781 - val_loss: 2.5725
Epoch 4/20
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 219s 125ms/step - accuracy: 0.3373 - loss: 2.2028 - val_accuracy: 0.2647 - val_loss: 2.6397
Epoch 5/20
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 264s 126ms/step - accuracy: 0.4097 - loss: 1.9440 - val_accuracy: 0.2609 - val_loss: 2.8351
Epoch 6/20
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 221s 127ms/step - accuracy: 0.4922 - loss: 1.6759 - val_accuracy: 0.2357 - val_loss: 2.9869
Epoch 7/20
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 212s 121ms/step - accuracy: 0.5778 - loss: 1.4004 - val_accuracy: 0.2171 - val_loss: 3.2494
Epoch 8/20
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 210s 121ms/step - ac

In [41]:
y_pred_probs =model.predict(X_test_pad,verbose=0)
y_pred_classes =np.argmax(y_pred_probs,axis=1)

In [42]:
output_df =pd.DataFrame({
    'id': test_ids,'Label': y_pred_classes
})
output_df.to_csv('predictions.csv',index=False)

In [45]:
for i in range(min(5,len(X_test_raw))):
    pred_emoji =emj_dict.get(y_pred_classes[i],'Unknown')
    print(f"Text: {X_test_raw[i]}")
    print(f"Emoji: {pred_emoji}\n")
model.save('emoji_pred_model.keras')
with open('toke.pkl','wb') as f:
    pickle.dump(tokenizer,f)
with open('prep.pkl','wb') as f:
    pickle.dump({'maxlen': maxlen,'emj_dict': emj_dict},f)

Text: thought this was cool repost get_repost colorview by shay_images
Emoji: 📸

Text: happy 4th corte madera parade everytownusa merica perry s on
Emoji: 🇺🇸

Text: luv or at least something close to it union hill richmond virginia
Emoji: ❤

Text: there s a slice of pie under that whipped cream houseofpies house of pies
Emoji: 😍

Text: thankyou for your thank you we adore you both plan on moreeeee hosting your wedding was
Emoji: ✨

